# Week Two
## At Home Activity: Running a GWAS

Genome-wide association studies allow us to identify variation (SNPs) present in one population but not in another. These populations usually consist of two groups where one group has a particular trait while the other does not. In this activity you have been given data from the 1000 genomes project.

This data contains genetic information on these individuals as well as the amount of coffee they consume.
We will analyse the data to identify if there is a genetic cause for increased (or decreased) coffee consumption!

In [ ]:
#Load Libraries
import hail as hl
hl.init()

In [ ]:
#Load other libaries
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

In [ ]:
#Read in the data!
gwasdata = hl.read_matrix_table('1kg.mt')

In [ ]:
#Lets look at the data
#our data is called gwasdata
#the .show() function shows us a sample of the data
#explanations of the data is contained in the booklet!
gwasdata.show()

In [ ]:
##load other data
otherdata = (hl.import_table('1kg_annotations.txt', impute=True)
         .key_by('Sample'))

In [ ]:
#use .show() to look at otherdata


In [ ]:
#Add phenotype data to gwasdata
gwasdata = gwasdata.annotate_cols(pheno = otherdata[gwasdata.s])

In [ ]:
#Quality Control
gwasdata = hl.sample_qc(gwasdata)
#Show plot
plot = hl.plot.histogram(gwasdata.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(plot)

In [ ]:
#Remove outliers if necessary
gwasdata = gwasdata.filter_cols((gwasdata.sample_qc.dp_stats.mean >= 4) & (gwasdata.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % gwasdata.count_cols())

In [ ]:
#GWAS
#caffeine
c_gwas = hl.linear_regression_rows(y=gwasdata.pheno.CaffeineConsumption,
                                 x=gwasdata.GT.n_alt_alleles(),
                                 covariates=[1.0])
#plot results
c_result = hl.plot.manhattan(c_gwas.p_value)
show(c_result)

In [ ]:
#Run it for purple hair
hair_gwas = hl.linear_regression_rows(y=,
                                 x=gwasdata.GT.n_alt_alleles(),
                                 covariates=[1.0])
#plot results
hair_result = hl.plot.manhattan(hair_gwas.p_value)
show(hair_result)